## Importing Libraries

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")


Solving environment: / 
  - anaconda/linux-64::openssl-1.1.1d-h7b6447c_2
  - defaults/linux-64::openssl-1.1.1d-h7b6447cdone

## Package Plan ##

  environment location: /home/alpha21/anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          92 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.21.0-py_0



geographiclib-1.50   | 34 KB     | ##################################### | 100% 
geopy-1.21.0         | 58 KB     | ##################################### | 100% 


## Scrape data from wikipedia page

In [3]:
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text


In [4]:
# beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')


In [5]:
data

'\n<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>List of postal codes of Canada: M - Wikipedia</title>\n<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"XptHlApAAEEAABcBbZIAAAAD","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":951325562,"wgRevisionId":951325562,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related l

In [6]:
soup


<!DOCTYPE html>

<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>List of postal codes of Canada: M - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"XptHlApAAEEAABcBbZIAAAAD","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":951325562,"wgRevisionId":951325562,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related lists"]

In [18]:
# create three lists to store table data
postalCodeList = []
boroughList = []
neighborhoodList = []

In [19]:
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text.rstrip('\n'))
        boroughList.append(cells[1].text.rstrip('\n'))
        neighborhoodList.append(cells[2].text.rstrip('\n')) # avoid new lines in neighborhood cell


In [20]:
df1 = pd.DataFrame({"PostalCode": postalCodeList,"Borough": boroughList,"Neighborhood": neighborhoodList})

df1.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


## Dropping "not assigned" cells

In [21]:
df2 = df1[df1.Borough != "Not assigned"].reset_index(drop=True)
df2.head()


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


# group by borough

In [23]:
df3 = df2.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
df3.head()


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [24]:
for index, row in df3.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
        
df3.head()



,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


## Creating required dataframe

In [25]:
column_names = ["PostalCode", "Borough", "Neighborhood"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(df3[df3["PostalCode"]==postcode], ignore_index=True)
    
test_df

,PostalCode,Borough,Neighborhood
0,M5G,Downtown Toronto,Central Bay Street
1,M2H,North York,Hillcrest Village
2,M4B,East York,Parkview Hill / Woodbine Gardens
3,M1J,Scarborough,Scarborough Village
4,M4G,East York,Leaside
5,M4M,East Toronto,Studio District
6,M1R,Scarborough,Wexford / Maryvale
7,M9V,Etobicoke,South Steeles / Silverstone / Humbergate / Jam...
8,M9L,North York,Humber Summit
9,M5V,Downtown Toronto,CN Tower / King and Spadina / Railway Lands / ...


## Shape

In [27]:
df3.shape

(103, 3)

# Part 2
## Loading Co-ordinates

In [28]:
df_cood = pd.read_csv('Geospatial_Coordinates.csv')
df_cood.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [29]:
df_cood.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
df_cood.head()


,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## Merge them together

In [31]:
df_new = df3.merge(df_cood, on="PostalCode", how="left")
df_new.head()


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## required dataframe

In [32]:
column_names = ["PostalCode", "Borough", "Neighborhood", "Latitude", "Longitude"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(df_new[df_new["PostalCode"]==postcode], ignore_index=True)
    
test_df


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
1,M2H,North York,Hillcrest Village,43.803762,-79.363452
2,M4B,East York,Parkview Hill / Woodbine Gardens,43.706397,-79.309937
3,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
4,M4G,East York,Leaside,43.709060,-79.363452
5,M4M,East Toronto,Studio District,43.659526,-79.340923
6,M1R,Scarborough,Wexford / Maryvale,43.750072,-79.295849
7,M9V,Etobicoke,South Steeles / Silverstone / Humbergate / Jam...,43.739416,-79.588437
8,M9L,North York,Humber Summit,43.756303,-79.565963
9,M5V,Downtown Toronto,CN Tower / King and Spadina / Railway Lands / ...,43.628947,-79.394420


## Coordinates of Toronto

In [34]:
address = 'Toronto'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))


The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


## Map of Toronto

In [36]:
toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_new['Latitude'],df_new['Longitude'],df_new['Borough'], df_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(toronto)  
    
toronto

## Filter Borough names

In [37]:
borough_names = list(df_new.Borough.unique())

b_toronto = []

for x in borough_names:
    if "toronto" in x.lower():
        b_toronto.append(x)
        
b_toronto

['East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto']

In [38]:
# only boroughs in toronto
df_new = df_new[df_new['Borough'].isin(b_toronto)].reset_index(drop=True)
df_new.head()


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,The Danforth West / Riverdale,43.679557,-79.352188
2,M4L,East Toronto,India Bazaar / The Beaches West,43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [39]:
#New map
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_new['Latitude'], df_new['Longitude'], df_new['Borough'], df_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

In [40]:
 #define Foursquare Credentials and Version
CLIENT_ID = 'G3QIXXNMDCLCSIOP2035KSDSR0VI50XTSOPHMLO0SDNQVA0H' # your Foursquare ID
CLIENT_SECRET = 'SZBAHCYCUUS5KJY0QGQM5WFKTI2NIEYXPHXMK3I1ZE5AZXBC' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: G3QIXXNMDCLCSIOP2035KSDSR0VI50XTSOPHMLO0SDNQVA0H
CLIENT_SECRET:SZBAHCYCUUS5KJY0QGQM5WFKTI2NIEYXPHXMK3I1ZE5AZXBC


# Exploring Torornto

In [42]:
radius = 1000
LIMIT = 100

venues = []

for lat, long, post, borough, neighborhood in zip(df_new['Latitude'], df_new['Longitude'], df_new['PostalCode'], df_new['Borough'], df_new['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))


In [43]:
 #convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['PostalCode', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()


(3190, 9)


,PostalCode,Borough,Neighborhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,M4E,East Toronto,The Beaches,43.676357,-79.293031,Tori's Bakeshop,43.672114,-79.290331,Vegetarian / Vegan Restaurant
2,M4E,East Toronto,The Beaches,43.676357,-79.293031,The Beech Tree,43.680493,-79.288846,Gastropub
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,The Fox Theatre,43.672801,-79.287272,Indie Movie Theater
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,Beaches Bake Shop,43.680363,-79.289692,Bakery


In [44]:
venues_df.groupby(["PostalCode", "Borough", "Neighborhood"]).count()


,,,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
PostalCode,Borough,Neighborhood,,,,,,
M4E,East Toronto,The Beaches,76,76,76,76,76,76
M4K,East Toronto,The Danforth West / Riverdale,100,100,100,100,100,100
M4L,East Toronto,India Bazaar / The Beaches West,81,81,81,81,81,81
M4M,East Toronto,Studio District,100,100,100,100,100,100
M4N,Central Toronto,Lawrence Park,10,10,10,10,10,10
M4P,Central Toronto,Davisville North,100,100,100,100,100,100
M4R,Central Toronto,North Toronto West,45,45,45,45,45,45
M4S,Central Toronto,Davisville,100,100,100,100,100,100
M4T,Central Toronto,Moore Park / Summerhill East,64,64,64,64,64,64


In [45]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))


There are 272 uniques categories.


In [46]:
venues_df['VenueCategory'].unique()[:50]


array(['Trail', 'Vegetarian / Vegan Restaurant', 'Gastropub',
       'Indie Movie Theater', 'Bakery', 'Ice Cream Shop',
       'Toy / Game Store', 'Bagel Shop', 'Health Food Store',
       'Coffee Shop', 'French Restaurant', 'Pub', 'Park',
       'Mexican Restaurant', 'Beach', 'Breakfast Spot', 'Nail Salon',
       'Juice Bar', 'Diner', 'Japanese Restaurant',
       'Mediterranean Restaurant', 'Liquor Store', 'Tea Room',
       'Cupcake Shop', 'Pharmacy', 'Bar', 'Indian Restaurant',
       'Caribbean Restaurant', 'Thai Restaurant', 'Grocery Store',
       'Sandwich Place', 'Burger Joint', 'Greek Restaurant', 'Café',
       'Ramen Restaurant', 'Chocolate Shop', 'Pizza Place',
       'Asian Restaurant', 'Beer Store', 'Bank', 'Restaurant',
       'Furniture / Home Store', 'Scenic Lookout', 'Salon / Barbershop',
       'Sporting Goods Shop', 'Tree', 'Shopping Mall',
       'Italian Restaurant', 'Cosmetics Shop', 'Brewery'], dtype=object)

## Analyze the city

In [47]:
# one hot encoding
toronto_1 = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add postal, borough and neighborhood column back to dataframe
toronto_1['PostalCode'] = venues_df['PostalCode'] 
toronto_1['Borough'] = venues_df['Borough'] 
toronto_1['Neighborhoods'] = venues_df['Neighborhood'] 

# move postal, borough and neighborhood column to the first column
fixed_columns = list(toronto_1.columns[-3:]) + list(toronto_1.columns[:-3])
toronto_1 = toronto_1[fixed_columns]

print(toronto_1.shape)
toronto_1.head()


(3190, 275)


,PostalCode,Borough,Neighborhoods,Accessories Store,Airport,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,College Rec Center,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Design Studio,Dessert Shop,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Residential Building (Apartment / Condo),Restaurant,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Stadium,Soup Place,South American Restaurant,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Storage Facility,Street Art,Supermarket,Sushi Restaurant,Syrian Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Tree,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Women's Store,Yoga Studio,Zoo
0,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [48]:
toronto_grouped = toronto_1.groupby(["PostalCode", "Borough", "Neighborhoods"]).mean().reset_index()

print(toronto_grouped.shape)
toronto_grouped

(39, 275)


,PostalCode,Borough,Neighborhoods,Accessories Store,Airport,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,College Rec Center,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Design Studio,Dessert Shop,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Residential Building (Apartment / Condo),Restaurant,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Stadium,Soup Place,South American Restaurant,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Storage Facility,Street Art,Supermarket,Sushi Restaurant,Syrian Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Tree,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Women's Store,Yoga Studio,Zoo
0,M4E,East Toronto,The Beaches,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.00000

In [49]:
# TOP 10 places grouped by postal code
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns = ['PostalCode', 'Borough', 'Neighborhoods']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode'] = toronto_grouped['PostalCode']
neighborhoods_venues_sorted['Borough'] = toronto_grouped['Borough']
neighborhoods_venues_sorted['Neighborhoods'] = toronto_grouped['Neighborhoods']

for ind in np.arange(toronto_grouped.shape[0]):
    row_categories = toronto_grouped.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]

# neighborhoods_venues_sorted.sort_values(freqColumns, inplace=True)
print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted


(39, 13)


,PostalCode,Borough,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,Pub,Coffee Shop,Pizza Place,Breakfast Spot,Japanese Restaurant,Beach,Bakery,Burger Joint,Bar,Tea Room
1,M4K,East Toronto,The Danforth West / Riverdale,Greek Restaurant,Café,Coffee Shop,Pub,Fast Food Restaurant,Pizza Place,Italian Restaurant,Furniture / Home Store,Ramen Restaurant,Ice Cream Shop
2,M4L,East Toronto,India Bazaar / The Beaches West,Indian Restaurant,Coffee Shop,Café,Restaurant,Grocery Store,Beach,Fast Food Restaurant,Park,Brewery,Pizza Place
3,M4M,East Toronto,Studio District,Coffee Shop,Bar,Café,American Restaurant,Vietnamese Restaurant,Diner,Brewery,Bakery,French Restaurant,Italian Restaurant
4,M4N,Central Toronto,Lawrence Park,Café,Coffee Shop,Bus Line,Bookstore,Gym / Fitness Center,Park,College Quad,College Gym,Trail,Zoo
5,M4P,Central Toronto,Davisville North,Coffee Shop,Italian Restaurant,Café,Restaurant,Dessert Shop,Sushi Restaurant,Fast Food Restaurant,Gym,Pizza Place,Pharmacy
6,M4R,Central Toronto,North Toronto West,Italian Restaurant,Skating Rink,Coffee Shop,Park,Restaurant,Sporting Goods Shop,Mexican Restaurant,Café,Diner,Clothing Store
7,M4S,Central Toronto,Davisville,Coffee Shop,Italian Restaurant,Sushi Restaurant,Café,Dessert Shop,Pizza Place,Middle Eastern Restaurant,Gym,Pub,Restaurant
8,M4T,Central Toronto,Moore Park / Summerhill East,Grocery Store,Italian Restaurant,Park,Coffee Shop,Gym,Thai Restaurant,Bagel Shop,Pharmacy,Playground,Pub
9,M4V,Central Toronto,Summerhill West / Rathnelly / South Hill / For...,Coffee Shop,Sushi Restaurant,Park,Thai Restaurant,Italian Restaurant,Grocery Store,Gym,Pub,Liquor Store,Café


# Clustering


In [50]:
k = 5

toronto_grouped_clustering = toronto_grouped.drop(["PostalCode", "Borough", "Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=k, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]


array([1, 1, 1, 1, 0, 4, 4, 4, 4, 4], dtype=int32)

In [52]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
toronto_merged = df_new.copy()

# add clustering labels
toronto_merged["Cluster Labels"] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.drop(["Borough", "Neighborhoods"], 1).set_index("PostalCode"), on="PostalCode")

print(toronto_merged.shape)
toronto_merged.head() 

(39, 16)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Pub,Coffee Shop,Pizza Place,Breakfast Spot,Japanese Restaurant,Beach,Bakery,Burger Joint,Bar,Tea Room
1,M4K,East Toronto,The Danforth West / Riverdale,43.679557,-79.352188,1,Greek Restaurant,Café,Coffee Shop,Pub,Fast Food Restaurant,Pizza Place,Italian Restaurant,Furniture / Home Store,Ramen Restaurant,Ice Cream Shop
2,M4L,East Toronto,India Bazaar / The Beaches West,43.668999,-79.315572,1,Indian Restaurant,Coffee Shop,Café,Restaurant,Grocery Store,Beach,Fast Food Restaurant,Park,Brewery,Pizza Place
3,M4M,East Toronto,Studio District,43.659526,-79.340923,1,Coffee Shop,Bar,Café,American Restaurant,Vietnamese Restaurant,Diner,Brewery,Bakery,French Restaurant,Italian Restaurant
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Café,Coffee Shop,Bus Line,Bookstore,Gym / Fitness Center,Park,College Quad,College Gym,Trail,Zoo


In [53]:
# sort the results by Cluster Labels
print(toronto_merged.shape)
toronto_merged.sort_values(["Cluster Labels"], inplace=True)
toronto_merged


(39, 16)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Café,Coffee Shop,Bus Line,Bookstore,Gym / Fitness Center,Park,College Quad,College Gym,Trail,Zoo
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Pub,Coffee Shop,Pizza Place,Breakfast Spot,Japanese Restaurant,Beach,Bakery,Burger Joint,Bar,Tea Room
24,M5R,Central Toronto,The Annex / North Midtown / Yorkville,43.672710,-79.405678,1,Café,Coffee Shop,Vegetarian / Vegan Restaurant,Italian Restaurant,Bakery,Restaurant,Grocery Store,Gym,Museum,Pub
25,M5S,Downtown Toronto,University of Toronto / Harbord,43.662696,-79.400049,1,Café,Coffee Shop,Restaurant,Bakery,Vegetarian / Vegan Restaurant,Bar,Bookstore,Mexican Restaurant,Pizza Place,Beer Bar
26,M5T,Downtown Toronto,Kensington Market / Chinatown / Grange Park,43.653206,-79.400049,1,Café,Bar,Vegetarian / Vegan Restaurant,Coffee Shop,Art Gallery,Yoga Studio,Mexican Restaurant,Grocery Store,Bakery,Vietnamese Restaurant
30,M6G,Downtown Toronto,Christie,43.669542,-79.422564,1,Korean Restaurant,Café,Coffee Shop,Grocery Store,Ice Cream Shop,Mexican Restaurant,Cocktail Bar,Indian Restaurant,Ethiopian Restaurant,Park
32,M6J,West Toronto,Little Portugal / Trinity,43.647927,-79.419750,1,Café,Restaurant,Bar,Vegetarian / Vegan Restaurant,Bakery,Coffee Shop,Cocktail Bar,Pizza Place,Italian Restaurant,Asian Restaurant
33,M6K,West Toronto,Brockton / Parkdale Village / Exhibition Place,43.636847,-79.428191,1,Café,Coffee Shop,Bar,Restaurant,Furniture / Home Store,Bakery,Gift Shop,Tibetan Restaurant,Sandwich Place,Supermarket
35,M6R,West Toronto,Parkdale / Roncesvalles,43.648960,-79.456325,1,Pizza Place,Coffee Shop,Café,Sushi Restaurant,Bar,Bakery,Eastern European Restaurant,Pub,Restaurant,Sandwich Place
36,M6S,West Toronto,Runnymede / Swansea,43.651571,-79.484450,1,Coffee Shop,Café,Pizza Place,Bakery,Pub,Italian Restaurant,Sushi Restaurant,Latin American Restaurant,Falafel Restaurant,Spa


In [56]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(k)
ys = [i+x+(i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, post, bor, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['PostalCode'], toronto_merged['Borough'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [57]:
#Cluster 1
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,0,Café,Coffee Shop,Bus Line,Bookstore,Gym / Fitness Center,Park,College Quad,College Gym,Trail,Zoo


In [58]:
#Cluster 2
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,1,Pub,Coffee Shop,Pizza Place,Breakfast Spot,Japanese Restaurant,Beach,Bakery,Burger Joint,Bar,Tea Room
24,Central Toronto,1,Café,Coffee Shop,Vegetarian / Vegan Restaurant,Italian Restaurant,Bakery,Restaurant,Grocery Store,Gym,Museum,Pub
25,Downtown Toronto,1,Café,Coffee Shop,Restaurant,Bakery,Vegetarian / Vegan Restaurant,Bar,Bookstore,Mexican Restaurant,Pizza Place,Beer Bar
26,Downtown Toronto,1,Café,Bar,Vegetarian / Vegan Restaurant,Coffee Shop,Art Gallery,Yoga Studio,Mexican Restaurant,Grocery Store,Bakery,Vietnamese Restaurant
30,Downtown Toronto,1,Korean Restaurant,Café,Coffee Shop,Grocery Store,Ice Cream Shop,Mexican Restaurant,Cocktail Bar,Indian Restaurant,Ethiopian Restaurant,Park
32,West Toronto,1,Café,Restaurant,Bar,Vegetarian / Vegan Restaurant,Bakery,Coffee Shop,Cocktail Bar,Pizza Place,Italian Restaurant,Asian Restaurant
33,West Toronto,1,Café,Coffee Shop,Bar,Restaurant,Furniture / Home Store,Bakery,Gift Shop,Tibetan Restaurant,Sandwich Place,Supermarket
35,West Toronto,1,Pizza Place,Coffee Shop,Café,Sushi Restaurant,Bar,Bakery,Eastern European Restaurant,Pub,Restaurant,Sandwich Place
36,West Toronto,1,Coffee Shop,Café,Pizza Place,Bakery,Pub,Italian Restaurant,Sushi Restaurant,Latin American Restaurant,Falafel Restaurant,Spa
3,East Toronto,1,Coffee Shop,Bar,Café,American Restaurant,Vietnamese Restaurant,Diner,Brewery,Bakery,French Restaurant,Italian Restaurant


In [59]:
#Cluster 3
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,Downtown Toronto,2,Coffee Shop,Hotel,Café,Japanese Restaurant,American Restaurant,Seafood Restaurant,Concert Hall,Gastropub,Restaurant,Bookstore
29,Downtown Toronto,2,Coffee Shop,Café,Hotel,Japanese Restaurant,Seafood Restaurant,Theater,Restaurant,Park,Thai Restaurant,Gym
28,Downtown Toronto,2,Coffee Shop,Hotel,Café,Seafood Restaurant,Japanese Restaurant,Gastropub,Restaurant,Park,Breakfast Spot,American Restaurant
20,Downtown Toronto,2,Hotel,Café,Coffee Shop,Japanese Restaurant,Concert Hall,Restaurant,Seafood Restaurant,Theater,Gym,American Restaurant
19,Downtown Toronto,2,Coffee Shop,Café,Hotel,Park,Japanese Restaurant,Gym,Brewery,Plaza,Theater,Scenic Lookout
16,Downtown Toronto,2,Coffee Shop,Café,Hotel,Japanese Restaurant,Park,Restaurant,Seafood Restaurant,Bakery,New American Restaurant,Creperie
15,Downtown Toronto,2,Coffee Shop,Café,Restaurant,Gastropub,Hotel,Seafood Restaurant,Theater,Italian Restaurant,Plaza,Bakery
14,Downtown Toronto,2,Coffee Shop,Japanese Restaurant,Gastropub,Café,Theater,Italian Restaurant,Restaurant,Seafood Restaurant,Plaza,Hotel
18,Downtown Toronto,2,Coffee Shop,Café,Theater,Japanese Restaurant,Hotel,American Restaurant,Restaurant,Italian Restaurant,Beer Bar,Tea Room
10,Downtown Toronto,2,Park,Coffee Shop,Grocery Store,Convenience Store,BBQ Joint,Playground,Office,Sandwich Place,Candy Store,Filipino Restaurant


In [60]:
#Cluster 4
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
27,Downtown Toronto,3,Coffee Shop,Café,Harbor / Marina,Sushi Restaurant,Sculpture Garden,Track,Park,Garden,Dog Run,Dance Studio


In [61]:
#Cluster 5
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,Downtown Toronto,4,Coffee Shop,Sushi Restaurant,Park,Gastropub,Ramen Restaurant,Italian Restaurant,Burger Joint,Japanese Restaurant,Restaurant,Thai Restaurant
5,Central Toronto,4,Coffee Shop,Italian Restaurant,Café,Restaurant,Dessert Shop,Sushi Restaurant,Fast Food Restaurant,Gym,Pizza Place,Pharmacy
6,Central Toronto,4,Italian Restaurant,Skating Rink,Coffee Shop,Park,Restaurant,Sporting Goods Shop,Mexican Restaurant,Café,Diner,Clothing Store
7,Central Toronto,4,Coffee Shop,Italian Restaurant,Sushi Restaurant,Café,Dessert Shop,Pizza Place,Middle Eastern Restaurant,Gym,Pub,Restaurant
8,Central Toronto,4,Grocery Store,Italian Restaurant,Park,Coffee Shop,Gym,Thai Restaurant,Bagel Shop,Pharmacy,Playground,Pub
9,Central Toronto,4,Coffee Shop,Sushi Restaurant,Park,Thai Restaurant,Italian Restaurant,Grocery Store,Gym,Pub,Liquor Store,Café
12,Downtown Toronto,4,Coffee Shop,Park,Japanese Restaurant,Italian Restaurant,Men's Store,Sushi Restaurant,Restaurant,Burger Joint,Bookstore,Gastropub
11,Downtown Toronto,4,Park,Japanese Restaurant,Gastropub,Restaurant,Diner,Café,Pet Store,Caribbean Restaurant,Steakhouse,Bakery
13,Downtown Toronto,4,Coffee Shop,Pub,Diner,Theater,Park,Café,Breakfast Spot,Restaurant,Italian Restaurant,Bakery
23,Central Toronto,4,Bank,Coffee Shop,Café,Park,Japanese Restaurant,Italian Restaurant,Sushi Restaurant,Trail,Burger Joint,Skating Rink
